###### Target Trackin Optimization
## Author: Michel Barbeau, Carleton University
## Version: March 6, 2024


\begin{equation}
\mathop{\mathsf{minimize}}_x f(x)\\
\text{subject to } c(x) \le b
\end{equation}

In [102]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.optimize import Bounds, LinearConstraint, NonlinearConstraint
from timeit import default_timer as timer

In [103]:
# number of instants
n = 10
maxpos=10
print("\nNumber of instants: ", n)
# generate target's random positions
rng = np.random.default_rng()
pos = rng.integers(low=0, high=maxpos, size=n)
pos = np.array([0, 7, 4, 3, 3, 3, 8, 6, 9, 6])
print("\nTarget positions: ", pos)


Number of instants:  10

Target positions:  [0 7 4 3 3 3 8 6 9 6]


### Linear constraints

In [104]:
# minimum position
vmin = 0
# maximum position
vmax = 10
C1 = Bounds(np.array(np.zeros(n))+vmin, np.array(np.zeros(n))+vmax)
print(C1)

Bounds(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.]))


### Non linear constraint

In [105]:
# A = np.identity(n) # n * n identity matrix
smin = 1 # chaser-target minimum distance
smax = 2 # chaser-target maximum distance
c = smin * np.ones(n)
print(c)
d = smax * np.ones(n)
con = lambda x : abs(x - pos)
print (d)
C2 = NonlinearConstraint(con, c, d)
print(C2)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


### Objective function
The optimization object is to minimize the acceleration, defined as the difference of speed at time $i$ and $i-1$:
\begin{equation}
\vert s_{i+1} - s_{i} \vert \text{\ for\ } i=0,\ldots,n-1.
\end{equation}
which is equal to the position differences:
\begin{equation}
\vert (x_{i+2} - x_{i+1}) - (x_{i+1} - x_{i}) \vert \text{\ for\ } i=0,\ldots,n-2.
\end{equation}
which can be rewritten as:
\begin{equation}
\vert x_{i+2} - 2 x_{i+1}+ x_{i} \vert \text{\ for\ } i=0,\ldots,n-2.
\end{equation}
which is equivelent to minimizing the quatity
\begin{equation}
\left( x_{i+2} - 2 x_{i+1}+ x_{i} \right)^2 \text{\ for\ } i=0,\ldots,n-2.
\end{equation}

In [108]:
#f = lambda x : sum([ (x[i+2]-2*x[i+1]+x[i])**2 for i in range(n-2)])
f = lambda x : sum([ abs(x[i+2]-2*x[i+1]+x[i]) for i in range(n-2)])
start = timer()
sol = opt.minimize(f, np.zeros(n), bounds=C1, constraints=(C2,))
end = timer()
print(end - start)
print(np.round(sol.x))

0.01800987499882467
[2. 5. 3. 1. 1. 2. 6. 5. 7. 5.]
